# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [43]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [44]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biont_template = """你是一位非常博学的生物学教授。
你擅长以简洁易懂的方式回答关于生物的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_template = """你是一位很专业的计算机学家。你擅长回答关于计算机方面的问题。
之所以如此出色，是因为你读过很多关于计算机方面的书籍并且会将难题分解成各个部分
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

chinese_template = """你是一位很研究汉语言很多年的专家。你擅长回答关于汉语言方面的问题。
你擅长专业的角度去回答关于汉语言的问题。
你之所以那么专业，是因为你阅读过十分多的汉语言书籍和知识

这是一个问题：
{input}"""

In [45]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biont_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_template,
    },
]

In [46]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1",api_key="sk-IrIgzFee3yL0i2ccD5Bb6820566843EcBcDd1b94B034D01a")

In [47]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [48]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [49]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [50]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [51]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言问题']


In [52]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题


In [53]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [54]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题

<< INPUT >>
{input}

<< OU

In [55]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [56]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.


黑体辐射是指物体因为受到热量而发出的电磁辐射，它的频率和强度与物体的温度有关。具体来说，黑体辐射是一种热辐射，它的频谱特征由普朗克定律和斯蒂芬-玻尔兹曼定律描述。在理想情况下，一个完美的黑体会吸收所有入射的电磁波，并以相同的强度和频率发出辐射。黑体辐射在物理学中具有重要的应用，例如在研究宇宙微波背景辐射和太阳光谱等方面。


In [57]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '第一个大于40的质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.


答案：43，因为43是大于40的最小质数，且43+1=44能被3整除。


In [58]:
router_chain.verbose = True

In [59]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.


黑洞是一种非常密集的天体，它的引力非常强大，甚至连光也无法逃离它的吸引力。它的形成是由于恒星耗尽燃料后塌缩而成，密度极高，使得其引力场非常强大。黑洞的边界被称为“事件视界”，在这一点以内，任何事物都无法逃离黑洞的吸引力。由于黑洞吞噬一切物质，所以我们无法直接观测它，但是它的存在可以通过观测其周围物质的运动和辐射来间接证明。黑洞是宇宙中非常神秘的天体，也是物理学研究的重要课题。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [60]:
print(chain.run("多细胞生物和胚胎发育是如何形成的?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '多细胞生物和胚胎发育是如何形成的?'}
> Finished chain.


多细胞生物的形成和胚胎发育是由细胞分裂、细胞迁移和细胞分化这些过程控制的。在受精卵形成后，细胞不断分裂，形成胚胎的各个细胞层。同时，一些细胞会迁移到特定的位置，形成不同的器官和组织。最后，细胞会根据其位置和功能的不同，发生分化，形成不同类型的细胞，从而构成完整的多细胞生物。这个过程非常复杂，受到许多因素的调控，包括基因表达、信号传导和环境因素等。


In [61]:
print(chain.run("简述《左传》的辞令艺术?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '简述《左传》的辞令艺术?'}
> Finished chain.
 《左传》是一部重要的史书，记录了春秋时期鲁国的政治、社会和文化变革。它的辞令艺术主要体现在其文字的精炼、表达的深刻和情感的真实。它使用的辞令多为古文，但也融入了一些口语化的表达，使其更加生动易懂。此外，它还运用了比喻、对偶、排比等修辞手法，使其文笔更加优美。总的来说，《左传》的辞令艺术是其独特的叙述方式和精湛的文字表达，让读者能够更深入地理解春秋时期的历史。


In [62]:
print(chain.run("react和vue的区别是什么?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'What is the difference between react and vue?'}
> Finished chain.


React和Vue之间有几个主要的区别，包括以下几个方面：

1. 构建方式：React是一个JavaScript库，而Vue是一个JavaScript框架。React主要关注构建用户界面，而Vue提供了更多的工具和功能来构建整个应用程序。

2. 数据绑定：React采用单向数据绑定，即数据流向只能从父组件到子组件。Vue采用双向数据绑定，即数据可以在父子组件之间双向流动。

3. 组件化：React使用JSX语法来构建组件，而Vue使用模板语法来构建组件。JSX语法更接近于JavaScript，而模板语法更接近于HTML。

4. 状态管理：React使用props和state来管理组件的状态。Vue则提供了Vuex这样的状态管理库来帮助管理应用程序的状态。

5. 虚拟DOM：React使用虚拟DOM来
